In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, BatchNorm, global_add_pool, ChebConv, global_max_pool, SAGPooling, GATConv, GATv2Conv, TransformerConv, SuperGATConv, global_mean_pool, Linear
from torch.nn import BatchNorm1d
from math import floor

# Models

In [3]:
class EEGGraphConvNet(nn.Module):
    
    def __init__(self, reduced_sensors=True, sfreq=None, batch_size=256):
        super(EEGGraphConvNet, self).__init__()
        # Define and initialize hyperparameters
        self.sfreq = sfreq
        self.batch_size = batch_size
        self.input_size = 8 if reduced_sensors else 62
        
        # Layers definition
        # Graph convolutional layers
        self.conv1 = GCNConv(-1, 16, cached=True, normalize=False)
        self.conv2 = GCNConv(16, 32, cached=True, normalize=False)
        self.conv3 = GCNConv(32, 64, cached=True, normalize=False)
        self.conv4 = GCNConv(64, 50, cached=True, normalize=False)
        
        # Batch normalization
        self.batch_norm = BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        # Fully connected layers
        self.fc1 = nn.Linear(50, 30)
        self.fc2 = nn.Linear(30, 20)
        self.fc3 = nn.Linear(20, 2)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)

    def forward(self, x, edge_index, edge_weigth, batch):
        # Perform all graph convolutions
        x = F.leaky_relu(self.conv1(x, edge_index, edge_weigth), negative_slope=0.01)
        #x = F.dropout(x, p=0.2, training=self.training)
        
        x = F.leaky_relu(self.conv2(x, edge_index, edge_weigth), negative_slope=0.01)
        #x = F.dropout(x, p=0.2, training=self.training)
        
        x = F.leaky_relu(self.conv3(x, edge_index, edge_weigth), negative_slope=0.01)
        #x = F.dropout(x, p=0.2, training=self.training)
        
        conv_out = self.conv4(x, edge_index, edge_weigth)
        # Perform batch normalization
        batch_norm_out = F.leaky_relu(self.batch_norm(conv_out), negative_slope=0.01)
        #x = F.dropout(batch_norm_out, p=0.2, training=self.training)
        
        # Global add pooling
        mean_pool = global_add_pool(batch_norm_out, batch=batch)
        
        # Apply fully connected layters
        out = F.leaky_relu(self.fc1(mean_pool), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc3(out))
        return out

In [4]:
class EEGGraphConvNetTemporal(nn.Module):
    
    def __init__(self, **kwargs):
        super(EEGGraphConvNetTemporal, self).__init__()
        # Layers definition
        input_size = kwargs.pop("input_size", 1280)        
        # Graph convolutional layers
        self.conv1 = GCNConv(input_size, 640, cached=True, normalize=False)
        self.conv2 = GCNConv(640, 512, cached=True, normalize=False)
        self.conv3 = GCNConv(512, 256, cached=True, normalize=False)
        self.conv4 = GCNConv(256, 256, cached=True, normalize=False)
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm3 = BatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm4 = BatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        # Fully connected layers
        self.fc1 = nn.Linear(256, 128)
        
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        x = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm2(self.conv2(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm3(self.conv3(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm4(self.conv4(x, edge_index, edge_weigth)), negative_slope=0.01)
        #out = F.dropout(x, p = 0.2, training=self.training)

        add_pool = global_add_pool(x, batch=batch)
        out = F.leaky_relu(self.fc1(add_pool), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        out = F.leaky_relu(self.fc3(out))
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        return out

In [5]:
class EEGGraphConvNetLSTM(nn.Module):
    
    def __init__(self, reduced_sensors=True, sfreq=None, batch_size=256, **kwargs):
        super(EEGGraphConvNetLSTM, self).__init__()
        # Define and initialize hyperparameters
        self.sfreq = sfreq
        self.batch_size = batch_size
        #self.input_size = 8 if reduced_sensors else 62
        
        # Layers definition
        input_size = kwargs.pop("input_size", 1280)
        hidden_dim = 320
        
        # Graph convolutional layers
        self.conv1 = GCNConv(1280, 640, cached=True, normalize=False)
        self.conv2 = GCNConv(640, 512, cached=True, normalize=False)
        self.conv3 = GCNConv(512, 256, cached=True, normalize=False)
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm3 = BatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm4 = BatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        self.lstm = nn.LSTM(256, 256, 1, dropout=0.2)
        # Fully connected layers
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        
        x = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm2(self.conv2(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm3(self.conv3(x, edge_index, edge_weigth)), negative_slope=0.01)
        x, _ = self.lstm(x)
        #x = F.leaky_relu(self.batch_norm4(self.conv4(x, edge_index, edge_weigth)), negative_slope=0.01)
        #x = F.dropout(batch_norm_out, p=0.2, training=self.training)
        
        # Global add pooling
        add_pool = global_add_pool(x, batch=batch)
        # Apply fully connected layters
        out = F.leaky_relu(self.fc1(add_pool), negative_slope=0.01)
        out = F.dropout(out, p = 0.2, training=self.training)
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        out = F.leaky_relu(self.fc3(out))
        return out
        

In [6]:
class EEGConvNetMini(nn.Module):
    """Same as EEGGraphConvNet but with fewer 
    convolutional layers
    """
    def __init__(self, reduced_sensors=True, sfreq=None, batch_size=256):
        super(EEGConvNetMini, self).__init__()
        # Define and initialize hyperparameters
        self.sfreq = sfreq
        self.batch_size = batch_size
        self.input_size = 8 if reduced_sensors else 62
        
        # Layers definition
        # Graph convolutional layers
        self.conv1 = GCNConv(-1, 16, cached=True, normalize=False)
        
        # Batch normalization
        self.batch_norm = BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        # Fully connected layers
        self.fc1 = nn.Linear(16, 8)
        self.fc2 = nn.Linear(8, 4)
        self.fc3 = nn.Linear(4, 2)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        # Perform all graph convolutions
        x = F.leaky_relu(self.conv1(x, edge_index, edge_weigth), negative_slope=0.01)
        #x = F.dropout(x, p=0.2, training=self.training)
        
        # Perform batch normalization
        x = F.leaky_relu(self.batch_norm(x), negative_slope=0.01)
        #x = F.dropout(batch_norm_out, p=0.2, training=self.training)
        
        # Global add pooling
        mean_pool = global_add_pool(x, batch=batch)
        
        # Apply fully connected layters
        out = F.leaky_relu(self.fc1(mean_pool), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc3(out))
        return out
    
    

In [7]:
class EEGConvNetMiniV2(nn.Module):
    """Same as EEGGraphConvNet but with fewer 
    convolutional layers
    """
    def __init__(self, **kwargs):
        super(EEGConvNetMiniV2, self).__init__()
        # Layers definition
        # Graph convolutional layers
        self.conv1 = GCNConv(-1, 32, cached=True, normalize=False) #16
        self.conv2 = GCNConv(32, 64, cached=True, normalize=False) #32
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        # Perform all graph convolutions
        x = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm2(self.conv2(x, edge_index, edge_weigth)), negative_slope=0.01)
        #x = F.dropout(x, p=0.2, training=self.training)
        #x = F.dropout(batch_norm_out, p=0.2, training=self.training)
        # Global add pooling
        mean_pool = global_add_pool(x, batch=batch)
        
        # Apply fully connected layters
        out = F.leaky_relu(self.fc1(mean_pool), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc3(out))
        return out
    

In [8]:
class EEGConvNetMiniV3(nn.Module):
    """Same as EEGGraphConvNet but with fewer 
    convolutional layers
    """
    def __init__(self, **kwargs):
        super(EEGConvNetMiniV3, self).__init__()
        # Layers definition
        # Graph convolutional layers
        self.conv1 = GCNConv(-1, 32, cached=True, normalize=False)
        self.conv2 = GCNConv(32, 64, cached=True, normalize=False)
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

        
        # Fully connected layers
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        x = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm2(self.conv2(x, edge_index, edge_weigth)), negative_slope=0.01)
        mean_pool = global_add_pool(x, batch=batch)
        out = F.leaky_relu(self.fc1(mean_pool), negative_slope=0.01)        
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        out = F.leaky_relu(self.fc3(out))
        return out
    


In [9]:
class EEGConvNetMiniV2Attention(nn.Module):
    """Same as EEGGraphConvNet but with fewer 
    convolutional layers
    """
    def __init__(self, **kwargs):
        super(EEGConvNetMiniV2Attention, self).__init__()
        # Layers definition
        # Graph convolutional layers
        conv_layers = {
            "gatconv": GATConv,
            "gatconv2": GATv2Conv,
            "transformer": TransformerConv,
            "super": SuperGATConv
        }
        
        AttentionConv = conv_layers[kwargs.get("attention_conv", "gatconv")]
        nheads = kwargs.get("nheads", 1)
        hidden_dim = kwargs.get("hidden_dim", 32)
        output_dim = kwargs.get("output_dim", 64)
        concat = kwargs.get("concat", True)
        nclasses = kwargs.get("nclasses", 2)
        
        self.conv1 = AttentionConv(-1, hidden_dim, heads=nheads, concat=concat) #16
        self.conv2 = AttentionConv(hidden_dim * nheads, output_dim, heads=nheads, concat=concat)
        self.conv3 = AttentionConv(output_dim * nheads, output_dim * nheads * 2, heads=nheads, concat=concat)
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(hidden_dim * nheads, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(output_dim * nheads, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm3 = BatchNorm(output_dim * nheads * 2 * nheads, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) #output_dim * nheads * 2 * 2
        
        # Fully connected layers
        self.fc1 = nn.Linear(output_dim * nheads * 2 * nheads, output_dim * nheads * 2)
        self.fc2 = nn.Linear(output_dim * nheads * 2, output_dim * nheads)
        self.fc3 = nn.Linear((output_dim * nheads), nclasses)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        # Perform all graph convolutions
        x = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm2(self.conv2(x, edge_index)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm3(self.conv3(x, edge_index)), negative_slope=0.01)

        #x = F.dropout(x, p=0.2, training=self.training)
        #x = F.dropout(batch_norm_out, p=0.2, training=self.training)
        
        # Global add pooling
        mean_pool = global_add_pool(x, batch=batch)
        #print(mean_pool.size())
        
        # Apply fully connected layters
        out = F.leaky_relu(self.fc1(mean_pool), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc3(out))
        return out

In [10]:
class MultiLevelConvNet(nn.Module):
    """Same as EEGGraphConvNet but with fewer 
    convolutional layers
    """
    def __init__(self, **kwargs):
        super(MultiLevelConvNet, self).__init__()
        # Layers definition
        # Graph convolutional layers
        self.conv1 = GCNConv(-1, 32, cached=True, normalize=False)
        self.conv2 = GCNConv(32, 32, cached=True, normalize=False)
        self.conv3 = GCNConv(32, 64, cached=True, normalize=False)
        
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm3 = BatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        
        self.fc1 = nn.Linear(32, 64)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 64)
        
        # Fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(192, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Linear(32, 2),
        )
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        x1 = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index, edge_weigth)), negative_slope=0.01)
        x2 = F.leaky_relu(self.batch_norm2(self.conv2(x1, edge_index, edge_weigth)), negative_slope=0.01)
        x3 = F.leaky_relu(self.batch_norm3(self.conv3(x2, edge_index, edge_weigth)), negative_slope=0.01)
        
        add_pool1 = global_add_pool(x1, batch=batch)
        add_pool2 = global_add_pool(x2, batch=batch)
        add_pool3 = global_add_pool(x3, batch=batch)
        
        out1 = F.leaky_relu(self.fc1(add_pool1), negative_slope=0.01)        
        out2 = F.leaky_relu(self.fc2(add_pool2), negative_slope=0.01)        
        out3 = F.leaky_relu(self.fc3(add_pool3), negative_slope=0.01)
        
        out = torch.cat((out1, out2, out3), dim=1)        
        out = self.classifier(out)
        return out

In [11]:
class MAGE(nn.Module):
    def __init__(self, **kwargs):
        super(MAGE, self).__init__()
        # Layers definition
        # Graph convolutional layers
        conv_layers = {
            "gatconv": GATConv,
            "gatconv2": GATv2Conv,
            "transformer": TransformerConv,
            "super": SuperGATConv
        }
        
        AttentionConv = conv_layers[kwargs.get("attention_conv", "gatconv")]
        nheads = kwargs.get("nheads", 1)
        hidden_dim = kwargs.get("hidden_dim", 32)
        output_dim = kwargs.get("output_dim", 64)
        concat = kwargs.get("concat", True)
        
        nclasses = kwargs.get("nclasses", 2)
        
        self.conv1 = AttentionConv(-1, hidden_dim, heads=nheads, concat=concat) #16
        self.conv2 = AttentionConv(hidden_dim * nheads, hidden_dim * nheads, heads=nheads, concat=concat)
        self.conv3 = AttentionConv(hidden_dim * nheads * nheads, hidden_dim * nheads * nheads, heads=nheads, concat=concat)
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(hidden_dim * nheads, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(hidden_dim * nheads * nheads, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm3 = BatchNorm(hidden_dim * nheads * nheads * nheads, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_dim * nheads, hidden_dim * nheads)
        self.fc2 = nn.Linear(output_dim * nheads, output_dim * nheads)
        self.fc3 = nn.Linear(output_dim * nheads , output_dim * nheads * nheads)
        
        self.classifier = nn.Sequential(
            nn.Linear((hidden_dim * nheads) + (hidden_dim * nheads * nheads) + (hidden_dim * nheads * nheads * nheads), 128),
            nn.LeakyReLU(),
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 2),
        )
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        # Perform all graph convolutions
        x1 = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index)), negative_slope=0.01)
        x2 = F.leaky_relu(self.batch_norm2(self.conv2(x1, edge_index)), negative_slope=0.01)
        x3 = F.leaky_relu(self.batch_norm3(self.conv3(x2, edge_index)), negative_slope=0.01)
        
        # Global add pooling
        add_pool1 = global_add_pool(x1, batch=batch)
        add_pool2 = global_add_pool(x2, batch=batch)
        add_pool3 = global_add_pool(x3, batch=batch)
        
        #out1 = F.leaky_relu(self.fc1(add_pool1), negative_slope=0.01)        
        #out2 = F.leaky_relu(self.fc2(add_pool2), negative_slope=0.01)        
        #out3 = F.leaky_relu(self.fc3(add_pool3), negative_slope=0.01)

        out = torch.cat((add_pool1, add_pool2, add_pool3), dim=1)        
        out = self.classifier(out)
        return out
    

In [1]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader
from sklearn.metrics import roc_auc_score, balanced_accuracy_score
from numpy import mean
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pathlib import Path
import os

In [3]:
data_list = []
monements_pearson_path = Path('graphs/moments_pearson/')
for file in os.listdir(monements_pearson_path):
    data = torch.load(monements_pearson_path.joinpath(file))
    data_list.append(data)

In [9]:
data_list[0].label

1

In [16]:
train_list, test_list = train_test_split(data_list, test_size=0.3, random_state=4744)


In [17]:
train_list

[Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=0),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=0),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=2),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=0),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=0),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=2),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=

In [18]:
test_list

[Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=0),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=2),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=0),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=0),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=2),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=0),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=0),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=2),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=2),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=2),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=

In [19]:
train_loader = DataLoader(train_list, batch_size=16)

In [20]:
test_loader = DataLoader(test_list, batch_size=16)

In [21]:
model = EEGConvNetMiniV2Attention(attention_conv="transformer", nheads=2, hidden_dim=16, output_dim=32, nclasses=3)

In [22]:
model = model.double()

In [98]:
test_list[0].edge_attr

array([[ 1.        ,  0.76153398,  0.47364111,  0.49625678,  0.57560999,
         0.63875954,  0.5465498 ,  0.3406173 ,  0.28532149,  0.24783629,
         0.41335199,  0.68326831,  0.50928091,  0.59655049,  0.3751361 ,
         0.35948346,  0.46662683,  0.5233104 ,  0.23928294],
       [ 0.76153398,  0.        ,  0.43991159,  0.72888121,  0.39031691,
         0.52095233,  0.3662653 ,  0.12544106,  0.10757305,  0.16773811,
         0.35530036,  0.6332677 ,  0.40574011,  0.51353854,  0.26981602,
         0.30603446,  0.41618273,  0.31112191,  0.37064108],
       [ 0.47364111,  0.43991159,  1.        ,  0.81268732,  0.05537708,
         0.366651  ,  0.28468176,  0.20794254,  0.35429072,  0.56324822,
         0.93098389,  0.51106258,  0.82676685,  0.38853327,  0.81152884,
         0.80536848,  0.8259073 , -0.00554757,  0.57646427],
       [ 0.49625678,  0.72888121,  0.81268732,  1.        ,  0.05352186,
         0.31294323,  0.18454596,  0.10057525,  0.20591791,  0.40854138,
         0.726

torch.Size([19, 1])

SVM

In [29]:
y_train = [data.label for data in train_list]
y_test = [data.label for data in test_list]

x_train = [[np.std(x) for x in data.edge_attr] for data in train_list]
x_test = [[np.std(x) for x in data.edge_attr] for data in test_list]


In [30]:
from sklearn.svm import SVC
from sklearn import metrics

In [31]:
model = SVC()

In [32]:
x_train

[[0.20014946274504558,
  0.12447249222945671,
  0.20206889417789395,
  0.10687509460745652,
  0.19429647331120448,
  0.15215177982627745,
  0.19300119925611323,
  0.091698260154082,
  0.19429114864575286,
  0.18561679429172143,
  0.1913816648136844,
  0.09775919874072861,
  0.12638639185120829,
  0.1944982151156955,
  0.09504071392195745,
  0.18936645029355975,
  0.21222153774759922,
  0.12046977651957445,
  0.19349146186275176],
 [0.21323692587623966,
  0.18275695353908863,
  0.22120212560049085,
  0.052151442105203505,
  0.22087262435985364,
  0.22103533490635666,
  0.054777814783846454,
  0.22206304483894965,
  0.21939344621051346,
  0.2089790476064607,
  0.2193549368985272,
  0.21715985454465808,
  0.221547393588728,
  0.22199551076942914,
  0.2222819489158725,
  0.21832652369948236,
  0.22111853656641395,
  0.22222618206463238,
  0.05498192738373229],
 [0.1761144976688873,
  0.18367847571793727,
  0.19799314464374299,
  0.2110707529098685,
  0.1421918288851274,
  0.159982574888101

In [33]:

model.fit(x_train, y_train)


SVC()

In [34]:
y_pred = model.predict(x_test)

In [35]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.17      0.29        12
           1       0.28      1.00      0.44         7
           2       0.00      0.00      0.00         8

    accuracy                           0.33        27
   macro avg       0.43      0.39      0.24        27
weighted avg       0.52      0.33      0.24        27



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

# Node features

In [47]:
y_train = [data.label for data in train_list]
y_test = [data.label for data in test_list]

x_train = [data.x[5] for data in train_list]
x_test = [data.x[5] for data in test_list]

In [48]:
model = SVC()

In [49]:
x_train

[array([ 2.11339934e-05,  6.66159354e-05,  4.43768285e-09, -8.20041280e+00,
         5.58208392e-02, -2.12391052e-02]),
 array([ 1.24992132e-04,  8.33498491e-04,  6.94719734e-07, -7.10161292e+00,
         7.67759604e-01,  9.15164056e+00]),
 array([-3.64692359e-06,  6.79034848e-05,  4.61088325e-09, -8.18739250e+00,
         2.76200788e-01,  2.03062794e-01]),
 array([ 2.27730484e-05,  7.42379007e-05,  5.51126590e-09, -8.13365462e+00,
         8.80167850e-01,  8.04961871e+00]),
 array([-3.26333894e-05,  6.33436299e-05,  4.01241545e-09, -8.24604488e+00,
        -1.11824856e-01,  2.74494088e-02]),
 array([ 2.05007424e-05,  6.10986913e-05,  3.73305007e-09,            -inf,
        -7.15941882e-02,  2.79303348e-02]),
 array([ 7.02959125e-05,  6.84787335e-05,  4.68933695e-09, -8.17454264e+00,
         4.26923682e-02,  5.24417393e-02]),
 array([ 2.58139689e-05,  6.52081087e-05,  4.25209744e-09, -8.22121934e+00,
        -1.26370575e-01,  9.44925941e-02]),
 array([-1.00885318e-05,  8.08517803e-05

In [50]:
model.fit(x_train, y_train)

ValueError: Input X contains infinity or a value too large for dtype('float64').